<a href="https://colab.research.google.com/github/SigneGitSand/Stanford_SkinScan/blob/main/SkinScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install necesary dependencies:

In [ ]:
!pip install streamlit
!pip install PIL
!pip install streamlit pyngrok
!pip install gradio

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


## Extract text to generate knowledge base

In [ ]:
!pip install PyMuPDF

In [ ]:
import fitz
print(fitz.__doc__)
print(fitz.__file__)

def extract_text_chunks(pdf_path, chunk_size=500):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()

    # Simple chunking by characters
    chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]
    return chunks

pdfs_for_knowledge = ["/content/cancer-org.pdf", "/content/MIA.pdf"]
knowledge_texts = []
for pdf in pdfs_for_knowledge:
    knowledge_texts.extend(extract_text_chunks(pdf))

PyMuPDF 1.25.5: Python bindings for the MuPDF 1.25.6 library (rebased implementation).
Python 3.11 running on linux (64-bit).

/usr/local/lib/python3.11/dist-packages/fitz/__init__.py


### Vectorize the knowledge text:

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Vectorize the extracted chunks
embeddings = model.encode(knowledge_texts, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

def get_top_k_chunks(text_note, k=2):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Embed the input note
    note_embedding = model.encode(text_note, convert_to_tensor=True, device=device)

    # Ensure knowledge embeddings are on the same device
    embeddings_tensor = torch.tensor(embeddings, device=device)

    # Compute cosine similarities
    similarities = util.cos_sim(note_embedding, embeddings_tensor)[0]

    # Get top-k most similar chunk indices
    top_k_idx = similarities.topk(k=k).indices

    # Return top-k chunks and scores
    top_chunks = [(knowledge_texts[i], float(similarities[i])) for i in top_k_idx]
    return top_chunks

In [ ]:
# test encoder
text = "My mole is hurting and is growing in a weird shape"

top_chunks = get_top_k_chunks(text)

print(top_chunks)

[('on’t fit these rules. It’s important to tell your doctor about any\nchanges or new spots on the skin, or growths that look different from the rest of your\nmoles.\nOther warning signs are:\nA sore that doesn’t heal\nG\nSpread of pigment from the border of a spot into surrounding skin\nG\nRedness or a new swelling beyond the border of the mole\nG\nChange in sensation, such as itchiness, tenderness, or pain\nG\nChange in the surface of a mole – scaliness, oozing, bleeding, or the appearance of\na lump or bump\n', 0.6775822043418884), (' that it might occur.\nNormal moles\nA normal mole is usually an evenly colored brown, tan, or black spot on the skin. It can\nbe either flat or raised. It can be round or oval. Moles are generally less than 6\nmillimeters (about ¼ inch) across (about the width of a pencil eraser). Some moles can\nbe present at birth, but most appear during childhood or young adulthood. New moles\nthat appear later in life should be checked by a doctor.\nOnce a mole has

<ipython-input-88-40ee1de80918>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_tensor = torch.tensor(embeddings, device=device)


In [ ]:
import requests
from getpass import getpass
from google.colab import userdata
import os

os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")

def generate_llm_diagnosis(user_note, label, chunk_1, chunk_2, model="mistralai/mistral-medium-3"):
    # Get API key from environment variable instead of interactive prompt
    api_key = os.environ.get("OPENROUTER_API_KEY")

    # If no API key is available, return a default message
    if not api_key:
        return f"API key not found. To enable detailed analysis, please set the OPENROUTER_API_KEY environment variable.\n\nBased on the image classification: This appears to be {label}. Please consult a healthcare professional for proper diagnosis."

    prompt = f"""You are a medical AI assistant helping a user who uploaded a photo of their skin lesion. The image classifier predicted the lesion is: {label}.

User notes:
{user_note}

Relevant medical knowledge:
1. {chunk_1}
2. {chunk_2}

Is this condition dangerous? What should the user do next?
"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "OpenRouter-Model": model
    }

    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful and medically accurate assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 200
    }

    try:
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        print(f"LLM API error: {e}")
        # Provide a fallback response if the API call fails
        danger_level = "potentially concerning" if label == "melanoma" else "likely benign"
        return f"Error getting detailed analysis. Based on the image classification: This appears to be {label}, which is {danger_level}. Please consult a healthcare professional for proper diagnosis."



In [ ]:
response = generate_llm_diagnosis(user_note=text, label="melanoma", chunk_1=top_chunks[0][0], chunk_2=top_chunks[1][0])
print(response)

Based on the user's notes and the image classifier's prediction, the lesion in question is suspected to be melanoma, a type of skin cancer that can be dangerous if not treated promptly. Here's what the user should do next:

1. **Seek Immediate Medical Attention**: Given that the mole is hurting and growing in a weird shape, it's crucial to consult a healthcare professional as soon as possible. These could be signs of melanoma, which is a serious condition.

2. **Consult a Dermatologist**: A dermatologist specializes in skin conditions and can provide a more accurate diagnosis. They may perform a biopsy to determine if the lesion is indeed melanoma.

3. **Biopsy**: If the dermatologist suspects melanoma, they will likely perform a biopsy. This involves removing a small sample of the mole for laboratory testing to confirm the diagnosis.

4. **Further Tests**: If melanoma is confirmed, further tests may be needed to determine the stage of the cancer. This could include imaging tests


In [ ]:
import gradio as gr
from torchvision import models, transforms
from PIL import Image
import torch.nn as nn
import torch

# Types of moles
class_names = ["benign", "nevus", "melanoma"]

# CNN to classify images
image_model = models.efficientnet_b0(pretrained=True)
image_model.classifier[1] = nn.Linear(image_model.classifier[1].in_features, len(class_names))  # match class count
image_model.eval()

# Transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # normalization
        std=[0.229, 0.224, 0.225]
    )
])

# Label-to-description mapping (can be expanded)
label_to_description = {
    "benign": "A non-cancerous skin lesion. Usually harmless and does not require treatment.",
    "nevus": "A common mole, generally benign but should be monitored for changes.",
    "melanoma": "Melanoma is a serious form of skin cancer that arises when pigment-producing cells mutate."
}

# CNN-based classification + LLM-like response
def classify_skin_lesion(image, text_note):
    if image is None:
        return {}, "Please upload an image for analysis", "", ""

    # Preprocess image
    img_tensor = transform(image).unsqueeze(0)

    # Predict
    with torch.no_grad():
        output = image_model(img_tensor)
        probs = torch.nn.functional.softmax(output, dim=1)
        pred_idx = torch.argmax(probs, dim=1).item()
        label = class_names[pred_idx]

    # Get top chunks based on user notes
    top_chunks = []
    if text_note and text_note.strip():
        top_chunks = get_top_k_chunks(text_note, k=2)
    else:
        # If no notes, use label as the search query
        top_chunks = get_top_k_chunks(f"Information about {label}", k=2)

    # Extract chunk texts
    chunk_1 = top_chunks[0][0] if len(top_chunks) > 0 else "No relevant information found."
    chunk_2 = top_chunks[1][0] if len(top_chunks) > 1 else "No additional information found."

    # Get LLM-generated response
    llm_response = generate_llm_diagnosis(text_note, label, chunk_1, chunk_2)

    return {label: float(probs[0][pred_idx])}, llm_response, chunk_1, chunk_2

# CSS for better styling
css = """
.gradio-container {max-width: 900px !important}
.disclaimer {color: red; font-weight: bold; text-align: center; margin: 20px 0;}
"""

# Gradio interface
with gr.Blocks(css=css) as interface:
    gr.Markdown("# Skin Lesion Analysis Tool")
    gr.Markdown("Upload an image of a skin lesion for analysis. Add any notes about symptoms or concerns.")

    with gr.Row():
        with gr.Column(scale=1):
            input_image = gr.Image(type="pil", label="Upload Skin Lesion Image")
            input_text = gr.Textbox(lines=3, label="Additional Notes (symptoms, concerns, etc.)")
            submit_btn = gr.Button("Analyze", variant="primary")

        with gr.Column(scale=1):
            output_class = gr.Label(label="Classification Result")
            output_llm = gr.Textbox(label="Medical Assessment", lines=8)

        with gr.Column("Relevant Medical Information"):
          chunk1 = gr.Textbox(label="Primary Reference", lines=10)
          chunk2 = gr.Textbox(label="Secondary Reference", lines=10)
    gr.Markdown("""<div class="disclaimer">DISCLAIMER: This tool is for educational purposes only and should not replace professional medical advice. Always consult a healthcare professional for proper diagnosis and treatment.</div>""")

    # Set up the submission action
    submit_btn.click(
        fn=classify_skin_lesion,
        inputs=[input_image, input_text],
        outputs=[output_class, output_llm, chunk1, chunk2]
    )

    gr.Markdown("### How to use")
    gr.Markdown("""
    1. Upload a clear, well-lit photo of the skin lesion
    2. Add any relevant notes about symptoms or concerns
    3. Click 'Analyze' to get results
    4. Review the assessment and consider following up with a healthcare professional
    """)


if __name__ == "__main__":
    print("To enable LLM functionality, set your OpenRouter API key as an environment variable:")
    print("export OPENROUTER_API_KEY='your_api_key_here'")

    interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


To enable LLM functionality, set your OpenRouter API key as an environment variable:
export OPENROUTER_API_KEY='your_api_key_here'
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af62e75578ed705905.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
